In [7]:
pip install beautifulsoup4

     |████████████████████████████████| 102kB 7.4MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [10]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

!conda install -c conda-forge folium=0.5.0 --yes
import folium
import json
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          90 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.20.0-py_0 conda-forge


geopy-1.20.0         | 57 KB     | ##################################### | 100% 
geographiclib-1.49   | 

Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [11]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
result = requests.get(url)
print(url)
print(result.status_code)
print(result.headers)

https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
200
{'Date': 'Thu, 08 Aug 2019 21:04:38 GMT', 'Content-Type': 'text/html; charset=UTF-8', 'Content-Length': '15023', 'Connection': 'keep-alive', 'Server': 'mw1270.eqiad.wmnet', 'X-Powered-By': 'PHP/7.2.16-1+0~20190307202415.17+stretch~1.gbpa7be82+wmf1', 'X-Content-Type-Options': 'nosniff', 'P3P': 'CP="This is not a P3P policy! See https://en.wikipedia.org/wiki/Special:CentralAutoLogin/P3P for more info."', 'Content-language': 'en', 'Last-Modified': 'Thu, 25 Jul 2019 12:38:22 GMT', 'Backend-Timing': 'D=113505 t=1565267902527560', 'Vary': 'Accept-Encoding,Cookie,Authorization,X-Seven', 'Content-Encoding': 'gzip', 'X-Varnish': '31861018 636622609, 24885423 1067414885', 'Age': '30375', 'X-Cache': 'cp1081 hit/5, cp1081 hit/1', 'X-Cache-Status': 'hit-front', 'Server-Timing': 'cache;desc="hit-front"', 'Strict-Transport-Security': 'max-age=106384710; includeSubDomains; preload', 'Set-Cookie': 'WMF-Last-Access=08-Aug-2019;Path=/;Ht

Get data and clean it

In [12]:
soup = BeautifulSoup(result.content, 'html.parser')
table = soup.find('table')
trs = table.find_all('tr')
rows = []
for tr in trs:
    i = tr.find_all('td')
    if i:
        rows.append(i)
        
lst = []
for row in rows:
    postalcode = row[0].text.rstrip()
    borough = row[1].text.rstrip()
    neighborhood = row[2].text.rstrip()
    if borough != 'Not assigned':
        if neighborhood == 'Not assigned':
            neighborhood = borough
        lst.append([postalcode, borough, neighborhood])

# lst

In [13]:
cols = ['PostalCode', 'Borough', 'Neighborhood']
df = pd.DataFrame(lst, columns=cols)
print(df.shape)
# df[df.duplicated(['PostalCode'], keep=False)] - this would have shown the duplicate PostalCodes

(211, 3)


In [14]:
df = df.groupby('PostalCode').agg(
    {
        'Borough':'first', 
        'Neighborhood': ', '.join,}
    ).reset_index()

check if 'M5A' example is done correctly

In [15]:
df.loc[df['PostalCode'] == 'M5A']

,PostalCode,Borough,Neighborhood
53,M5A,Downtown Toronto,"Harbourfront, Regent Park"


Use the .shape method to print the number of rows of your dataframe

In [27]:
df.head(12)


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [28]:
df.shape

(103, 3)

In [20]:
!wget -O Geospatial_Coordinates.csv https://cocl.us/Geospatial_data
    

--2019-08-08 21:24:39--  https://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 169.48.113.201
Connecting to cocl.us (cocl.us)|169.48.113.201|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-08-08 21:24:39--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.26.197, 107.152.27.197
Connecting to ibm.box.com (ibm.box.com)|107.152.26.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-08-08 21:24:40--  https://ibm.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Reusing existing connection to ibm.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.ent.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [followin

In [21]:
dfgeo = pd.read_csv("Geospatial_Coordinates.csv")
dfgeo.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)

In [22]:
dfgeo.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


merge 2 dataframes based on a PostalCode column

In [53]:
df2 = pd.merge(df, dfgeo, on="PostalCode", how='left')

In [54]:
df2.head(20)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


check if the example returns the expected value(s)

In [56]:
df2.loc[df2['PostalCode'] == 'M5G']

,PostalCode,Borough,Neighborhood,Latitude,Longitude
57,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383


In [57]:
df2.set_index('PostalCode', inplace = True) 

Explore and cluster the neighborhoods in Toronto

In [58]:

address = 'Toronto'

geolocator = Nominatim(user_agent="coursera_hw")

try:
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    print('The geograpical coordinates of {} are {}, {}.'.format(address, latitude, longitude))
except AttributeError:
    print('Cannot find: {}, will drop index: {}'.format(address, index))


The geograpical coordinates of Toronto are 43.653963, -79.387207.


Create a map of Toronto with neighborhoods superimposed on top.

In [34]:
my_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df2['Latitude'], df2['Longitude'], df2['PostalCode']):
    label = folium.Popup(label)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(my_map)  
    
my_map

Define Foursquare Credentials and Version

In [59]:
CLIENT_ID = 'NJBI1NENJYENG0SGJTTS1RFHOKSQEHBMWXQHPRLSLI0XU0OX' # your Foursquare ID
CLIENT_SECRET = 'E5GJDJ4JT02EDEH0PVGKF1ZN0NJCRTLXPJFMYPP0OJ4XAKLA' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: NJBI1NENJYENG0SGJTTS1RFHOKSQEHBMWXQHPRLSLI0XU0OX
CLIENT_SECRET:E5GJDJ4JT02EDEH0PVGKF1ZN0NJCRTLXPJFMYPP0OJ4XAKLA


Let's explore the first neighborhood (DT Toronto) in our dataframe.

In [64]:
neighborhood_latitude = df2.loc['M5G']['Latitude']
neighborhood_longitude = df2.loc['M5G']['Longitude']


In [65]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=NJBI1NENJYENG0SGJTTS1RFHOKSQEHBMWXQHPRLSLI0XU0OX&client_secret=E5GJDJ4JT02EDEH0PVGKF1ZN0NJCRTLXPJFMYPP0OJ4XAKLA&v=20180605&ll=43.6579524,-79.3873826&radius=500&limit=100'

In [66]:
results = requests.get(url).json()
# results

In [67]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [68]:
venues = results['response']['groups'][0]['items']

In [69]:

nearby_venues = json_normalize(venues) # flatten JSON
nearby_venues

,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.location.address,venue.location.crossStreet,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,...,venue.location.cc,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.categories,venue.photos.count,venue.photos.groups,venue.location.neighborhood,venue.venuePage.id
0,e-0-537d4d6d498ec171ba22e7fe-0,0,"[{'summary': 'This spot is popular', 'type': '...",537d4d6d498ec171ba22e7fe,Jimmy's Coffee,82 Gerrard Street W,Gerrard & LaPlante,43.658421,-79.385613,"[{'label': 'display', 'lat': 43.65842123574496...",...,CA,Toronto,ON,Canada,"[82 Gerrard Street W (Gerrard & LaPlante), Tor...","[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",0,[],NaN,NaN
1,e-0-557231e3498e540f05f3083c-1,0,"[{'summary': 'This spot is popular', 'type': '...",557231e3498e540f05f3083c,Tim Hortons,70 Gerrard St West,Bay St,43.658570,-79.385123,"[{'label': 'display', 'lat': 43.65856999999997...",...,CA,Toronto,ON,Canada,"[70 Gerrard St West (Bay St), Toronto ON M5G 1...","[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",0,[],NaN,NaN
2,e-0-5a81ae339deb7d369fa7f146-2,0,"[{'summary': 'This spot is popular', 'type': '...",5a81ae339deb7d369fa7f146,Hailed Coffee,44 Gerrard St W,Yonge St,43.658833,-79.383684,"[{'label': 'display', 'lat': 43.65883296982352...",...,CA,Toronto,ON,Canada,"[44 Gerrard St W (Yonge St), Toronto ON M5G, C...","[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",0,[],College Park,NaN
3,e-0-4a8355bff964a520d3fa1fe3-3,0,"[{'summary': 'This spot is popular', 'type': '...",4a8355bff964a520d3fa1fe3,Mercatto,101 College St,NaN,43.660391,-79.387664,"[{'label': 'display', 'lat': 43.66039091189854...",...,CA,Toronto,ON,Canada,"[101 College St, Toronto ON M5G, Canada]","[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",0,[],NaN,NaN
4,e-0-4ab2b0b9f964a520e56b20e3-4,0,"[{'summary': 'This spot is popular', 'type': '...",4ab2b0b9f964a520e56b20e3,The Queen and Beaver Public House,35 Elm St.,at Bay St.,43.657472,-79.383524,"[{'label': 'display', 'lat': 43.65747228208784...",...,CA,Toronto,ON,Canada,"[35 Elm St. (at Bay St.), Toronto ON M5G 1H1, ...","[{'id': '4bf58dd8d48988d155941735', 'name': 'G...",0,[],NaN,NaN
5,e-0-539c6f13498e06f4cc765165-5,0,"[{'summary': 'This spot is popular', 'type': '...",539c6f13498e06f4cc765165,The Elm Tree Restaurant,43 Elm St,Bay,43.657397,-79.383761,"[{'label': 'display', 'lat': 43.65739749535259...",...,CA,Toronto,ON,Canada,"[43 Elm St (Bay), Toronto ON M5G 1H1, Canada]","[{'id': '52e81612bcbc57f1066b79f9', 'name': 'M...",0,[],NaN,88534436
6,e-0-4c8facf91664b1f79c90aa2f-6,0,"[{'summary': 'This spot is popular', 'type': '...",4c8facf91664b1f79c90aa2f,College Park Area,College St.,NaN,43.659751,-79.384911,"[{'label': 'display', 'lat': 43.65975136823981...",...,CA,Toronto,ON,Canada,"[College St., Toronto ON, Canada]","[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",0,[],NaN,NaN
7,e-0-4ae7b27df964a52068ad21e3-7,0,"[{'summary': 'This spot is popular', 'type': '...",4ae7b27df964a52068ad21e3,Japango,122 Elizabeth St.,at Dundas St. W,43.655268,-79.385165,"[{'label': 'display', 'lat': 43.65526771691681...",...,CA,Toronto,ON,Canada,"[122 Elizabeth St. (at Dundas St. W), Toronto ...","[{'id': '4bf58dd8d48988d1d2941735', 'name': 'S...",0,[],NaN,NaN
8,e-0-56ccd5cfcd1069ca160a797e-8,0,"[{'summary': 'This spot is popular', 'type': '...",56ccd5cfcd1069ca160a797e,Tsujiri,147 Dundas St W,at Elizabeth St,43.655374,-79.385354,"[{'label': 'display', 'lat': 43.65537430780922...",...,CA,Toronto,ON,Canada,"[147 Dundas St W (at Elizabeth St), Toronto ON...","[{'id': '4bf58dd8d48988d1dc931735', 'name': 'T...",0,[],NaN,NaN
9,e-0-504bbf2ce4b0168121235cbe-9,0,"[{'summary': 'This spot is popular', 'type': '...",504bbf2ce4b0168121235cbe,Sansotei Ramen 三草亭,179 Dundas St. W,btwn Centre Ave. & Chestnut St.,43.655157,-79.386501,"[{'label': 'display', 'lat': 43.65515746756124...",...,CA,Toronto,ON,Canada,[179 Dundas St. W (btwn Ce

In [70]:
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Jimmy's Coffee,Coffee Shop,43.658421,-79.385613
1,Tim Hortons,Coffee Shop,43.658570,-79.385123
2,Hailed Coffee,Coffee Shop,43.658833,-79.383684
3,Mercatto,Italian Restaurant,43.660391,-79.387664
4,The Queen and Beaver Public House,Gastropub,43.657472,-79.383524


In [71]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

82 venues were returned by Foursquare.


In [72]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [73]:
df2

,Borough,Neighborhood,Latitude,Longitude
PostalCode,,,,
M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
M1G,Scarborough,Woburn,43.770992,-79.216917
M1H,Scarborough,Cedarbrae,43.773136,-79.239476
M1J,Scarborough,Scarborough Village,43.744734,-79.239476
M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476


In [74]:
venues = getNearbyVenues(names=df2['Borough'],latitudes=df2['Latitude'],longitudes=df2['Longitude'])

Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
East York
East York
East Toronto
East York
East York
East York
East Toronto
East Toronto
East Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
North York
Central Toronto
Central Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
North York
North York
York
York
Downtown Toronto
Wes

In [75]:
print(venues.shape)
venues.head()

(2250, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Scarborough,43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,Scarborough,43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,Scarborough,43.784535,-79.160497,RIGHT WAY TO GOLF,43.785177,-79.161108,Golf Course
3,Scarborough,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
4,Scarborough,43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum


In [76]:
venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Central Toronto,109,109,109,109,109,109
Downtown Toronto,1281,1281,1281,1281,1281,1281
East Toronto,121,121,121,121,121,121
East York,81,81,81,81,81,81
Etobicoke,78,78,78,78,78,78
Mississauga,11,11,11,11,11,11
North York,241,241,241,241,241,241
Queen's Park,40,40,40,40,40,40
Scarborough,91,91,91,91,91,91


In [77]:
print('There are {} unique categories.'.format(len(venues['Venue Category'].unique())))

There are 279 unique categories.


In [78]:
# one hot encoding
onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['Neighborhood'] = venues['Neighborhood'] 

onehot.head()

,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [79]:
onehot.shape

(2250, 279)

In [80]:

grouped = onehot.groupby('Neighborhood').mean().reset_index()
grouped

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Central Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.018349,...,0.009174,0.000000,0.000000,0.009174,0.000000,0.000000,0.000000,0.000000,0.000000,0.009174
1,Downtown Toronto,0.000000,0.000781,0.000781,0.000781,0.000781,0.001561,0.002342,0.001561,0.014052,...,0.013271,0.002342,0.000000,0.004684,0.000000,0.006245,0.000000,0.000781,0.000000,0.002342
2,East Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.024793,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.024793
3,East York,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.012346,0.000000,0.000000,0.000000,0.000000,0.012346
4,Etobicoke,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012821,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012821,0.000000,0.000000
5,Mississauga,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,North York,0.004149,0.000000,0.004149,0.000000,0.000000,0.000000,0.000000,0.000000,0.008299,...,0.000000,0.004149,0.004149,0.008299,0.000000,0.000000,0.000000,0.004149,0.020747,0.000000
7,Queen's Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.025000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.025000,0.000000,0.025000
8,Scarborough,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010989,...,0.000000,0.000000,0.000000,0.010989,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,West Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.011236,0.000000,0.000000,0.011236,0.000000,0.005618,0.005618,0.000000,0.000000,0.005618


In [81]:
grouped.shape

(11, 279)

In [82]:
num_top_venues = 5

for hood in grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = grouped[grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Central Toronto----
            venue  freq
0     Coffee Shop  0.08
1            Park  0.06
2  Sandwich Place  0.06
3      Restaurant  0.05
4            Café  0.05


----Downtown Toronto----
         venue  freq
0  Coffee Shop  0.09
1         Café  0.05
2       Bakery  0.03
3   Restaurant  0.03
4        Hotel  0.03


----East Toronto----
                venue  freq
0         Coffee Shop  0.07
1    Greek Restaurant  0.07
2  Italian Restaurant  0.05
3      Ice Cream Shop  0.04
4             Brewery  0.03


----East York----
          venue  freq
0   Coffee Shop  0.06
1  Burger Joint  0.05
2   Pizza Place  0.05
3          Park  0.05
4          Bank  0.04


----Etobicoke----
                  venue  freq
0           Pizza Place  0.12
1        Sandwich Place  0.06
2              Pharmacy  0.05
3           Coffee Shop  0.05
4  Fast Food Restaurant  0.04


----Mississauga----
                      venue  freq
0               Coffee Shop  0.18
1                     Hotel  0.18
2  Mediterra

In [83]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = grouped['Neighborhood']

for ind in np.arange(grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Coffee Shop,Sandwich Place,Park,Café,Restaurant,Pizza Place,Dessert Shop,Sushi Restaurant,Pub,Clothing Store
1,Downtown Toronto,Coffee Shop,Café,Restaurant,Hotel,Bakery,Italian Restaurant,Bar,Japanese Restaurant,Park,Seafood Restaurant
2,East Toronto,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Brewery,Café,Pub,Bakery,Pizza Place,Park
3,East York,Coffee Shop,Park,Pizza Place,Burger Joint,Sandwich Place,Bank,Sporting Goods Shop,Pharmacy,Liquor Store,Pet Store
4,Etobicoke,Pizza Place,Sandwich Place,Coffee Shop,Pharmacy,Pool,Fast Food Restaurant,Grocery Store,Gym,Park,Fried Chicken Joint
5,Mississauga,Coffee Shop,Hotel,Gym / Fitness Center,Mediterranean Restaurant,Fried Chicken Joint,Burrito Place,Middle Eastern Restaurant,American Restaurant,Sandwich Place,Dumpling Restaurant
6,North York,Coffee Shop,Fast Food Restaurant,Clothing Store,Japanese Restaurant,Grocery Store,Park,Restaurant,Bank,Pizza Place,Women's Store
7,Queen's Park,Coffee Shop,Diner,Gym,Park,Café,Italian Restaurant,Japanese Restaurant,Spa,Smoothie Shop,Seafood Restaurant
8,Scarborough,Coffee Shop,Breakfast Spot,Chinese Restaurant,Fast Food Restaurant,Bakery,Pizza Place,Thai Restaurant,Skating Rink,Playground,Pharmacy
9,West Toronto,Bar,Coffee Shop,Café,Italian Restaurant,Pizza Place,Bakery,Restaurant,Breakfast Spot,Music Venue,Asian Restaurant


In [84]:
# set number of clusters
kclusters = 5

grouped_clustering = grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([3, 1, 1, 3, 3, 0, 3, 2, 3, 1], dtype=int32)

In [85]:
merged = grouped

# add clustering labels
merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
merged = merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

merged.head() # check the last columns!

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.018349,...,Coffee Shop,Sandwich Place,Park,Café,Restaurant,Pizza Place,Dessert Shop,Sushi Restaurant,Pub,Clothing Store
1,Downtown Toronto,0.0,0.000781,0.000781,0.000781,0.000781,0.001561,0.002342,0.001561,0.014052,...,Coffee Shop,Café,Restaurant,Hotel,Bakery,Italian Restaurant,Bar,Japanese Restaurant,Park,Seafood Restaurant
2,East Toronto,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.024793,...,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Brewery,Café,Pub,Bakery,Pizza Place,Park
3,East York,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,Coffee Shop,Park,Pizza Place,Burger Joint,Sandwich Place,Bank,Sporting Goods Shop,Pharmacy,Liquor Store,Pet Store
4,Etobicoke,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012821,...,Pizza Place,Sandwich Place,Coffee Shop,Pharmacy,Pool,Fast Food Restaurant,Grocery Store,Gym,Park,Fried Chicken Joint
